# Synchronizing with Real Time

## Simple use of `tclab.clock()`

The tclab module includes a function `clock` for synchronizing calculations with real time.  `clock(tperiod)` is an iterator that generates a sequence of equally spaced time steps from zero to `tperiod`  separated by one second intervals. For each step `clock` returns time since start rounded to the nearest 10th of a second.

In [10]:
import tclab

tperiod = 5
for t in tclab.clock(tperiod):
    print(t, "sec.")

0 sec.
1.0 sec.
2.0 sec.
3.0 sec.
4.0 sec.
5.0 sec.


`tclab.clock()` is implemented as a Python generator using the Python library `time.sleep()` function. A consequence of this implementation is that `tclab.clock()` is 'blocking' which limits its use for creating interactive demonstrations. See later sections of this user's guide for non-blocking alternatives that can be used for interactive demonstrations or GUI's.

## Optional Parameters

### `tstep`: Clock time step

An optional parameter `tstep` specifies a time step different from one second.

In [7]:
import tclab

tperiod = 5
tstep = 2.5
for t in tclab.clock(tperiod,tstep):
    print(t, "sec.")

0 sec.
2.5 sec.
5.0 sec.


### `tol`: Clock tolerance

There are some considerations when using `clock`. First, by its nature Python is not a real-time environment. `clock` makes a best effort to stay in sync with the wall clock but there can be no guarantees. The default behavior of `clock` is to maintain long-term synchronization with the real time clock. A `RuntimeError` is raised if the difference between clock time and real time is greater than the optional parameter `tol` (default value of 0.25).

The following cell demonstrates the use of `tol` to avoid run time errors due to an intermittent calculation that exceeds the time step specified by `tstep`. In this instance, an extra `sleep` timeout of 0.2 seconds occurs at t=1 and t=2. Together these cause the clock to be delayed a total of 0.4 seconds relative to real time. The default tolerance of 0.25 second would cause a run time error to be generated. Setting tol=0.5 avoids raising that exception.

In [15]:
import tclab
import time

tfinal = 5
tstep = 1

for t in tclab.clock(tfinal, tstep, tol=0.5):
    print(t, "sec.")
    if 0.5 < t < 2.5:
        time.sleep(1.2)

0 sec.
1.0 sec.
2.2 sec.
3.4 sec.
4.0 sec.
5.0 sec.


### Fixing run time errors due to loss of clock synchronization.

If you encounter a `RuntimeError` due to loss of synchronization, there are normally two remedies depending on the underlying cause:

1. Synchronization can be lost if you attempting to multitask (say do some web browsing) while an experiment is underway. In this case you should increase the `tol` to a larger value.
2. Synchronization can be lost if the loop includes lengthy calculations, such as a computationally intensive control calculation. In the case it may be necessary to increase the `tstep` parameter.

## Using `tclab.clock()` with TCLab

The `tclab.clock()` generator can be used to implement and test control and estimation algorithms using the the Temperature Control Laboratory.

In [11]:
import tclab

tfinal = 20
tstep = 2

with tclab.TCLab() as lab:
    lab.Q1(100)
    lab.Q2(100)
    print("\nSet Heater 1 to {0:f} %".format(lab.Q1()))
    print("Set Heater 2 to {0:f} %".format(lab.Q2()))

    sfmt = "   {0:5.1f} sec:   T1 = {1:0.1f} °C    T2 = {2:0.1f} °C"
    for t in tclab.clock(tfinal, tstep):
        print(sfmt.format(t, lab.T1, lab.T2), flush=True)

Arduino Leonardo connected on port /dev/cu.usbmodemWUAR1 at 115200 baud.
TCLab Firmware 1.3.0 Arduino Leonardo/Micro.

Set Heater 1 to 100.000000 %
Set Heater 2 to 100.000000 %
     0.0 sec:   T1 = 23.2 °C    T2 = 21.9 °C
     2.0 sec:   T1 = 23.2 °C    T2 = 20.6 °C
     4.0 sec:   T1 = 23.2 °C    T2 = 20.6 °C
     6.0 sec:   T1 = 23.2 °C    T2 = 20.6 °C
     8.0 sec:   T1 = 23.5 °C    T2 = 20.9 °C
    10.0 sec:   T1 = 23.5 °C    T2 = 20.9 °C
    12.0 sec:   T1 = 23.8 °C    T2 = 20.9 °C
    14.0 sec:   T1 = 24.4 °C    T2 = 22.2 °C
    16.0 sec:   T1 = 24.4 °C    T2 = 22.2 °C
    18.0 sec:   T1 = 25.1 °C    T2 = 22.5 °C
    20.0 sec:   T1 = 25.4 °C    T2 = 22.8 °C
TCLab disconnected successfully.
